# Optimized energy scenarios Giudicarie Esteriori

Author: Michele Urbani ([murbani@fbk.eu](mailto:murbani@fbk.eu))

In this notebook, we replicate the study in {cite:ps}`MAHBUB2016236`.

## Problem description

### Decision variables

The decision variables are:

1. **PV capacity**: the amount of installed PV capacity is 5 MW and it is
as the lower bound for the variable, whereas the calculated maximum PV capacity
is 42 MW, which is the upper bound.
2. **Heat production technologies**: individual wood, oil, LPG boilers, and
gound source heat pumps are decision variables expressed as percentages of the
total.
3. **Wood organic ranking cycle micro cogeneration** provides both thermal and
electrical power

### Constraints

The are two constraints: the first concerns the variables at points 2 and 3 of
the list above, which must sum to 1. The second constraints limits the total
wood consumption to be less than 57 GWh/year.

### Optimization objectives

There are four optimization objectives.

1. **CO$_2$ minimization**: the value of produced CO$_2$ is
``CO2-emission (corrected)`` in EnergyPLAN output.
2. **Annual cost minimization**: the annual cost is the sum of the annual
investment cost, variable operational and maintenance (O&M) cost, fixed
operational and maintenacne cost, and the variable O&M and fixed O&M costs.
3. **Load following capacity (LFC) minimization**: the LFC expresses how much
electricity production follows electrivity demand over a period (yearly in this
case).
4. **Energy system dependency (ESD) minimization** concerns the reduction of
foreign energy import.

## Problem declaration

The ``_evaluate`` method is analyzed in the following.

In [2]:
import inspect
from IPython.display import display, Markdown

from moea.models import get_model
from moea.algorithms import get_algorithm

model_name = 'Giudicarie'
algorithm_name = 'NSGA-II'

model = get_model(model_name)
algorithm = get_algorithm(algorithm_name, pop_size=100)

eval_code = inspect.getsource(model._evaluate)
display(Markdown("```python\n" + eval_code + "```"))

```python
    def _evaluate(self, x, out, *args, **kwargs):
        # Use enums for the percentage variables
        PV = 0
        OIL = 1
        LPG = 2
        BIOMASS = 3
        CHP = 4
        HP = 5

        z = np.sort(x[:, OIL:]).T

        percentages = np.zeros_like(x.T)
        percentages[PV] = x[:, PV]
        percentages[OIL] = z[OIL]
        percentages[LPG] = z[LPG] - z[OIL]
        percentages[BIOMASS] = z[BIOMASS] - z[LPG]
        percentages[CHP] = z[CHP] - z[BIOMASS]
        percentages[HP] = 1 - z[CHP]

        # Dump the input vectors to files
        for i, ind in enumerate(percentages.T):
            dump_input({k: v for k, v in zip(self.vars.index, ind)},
                       i, self.default_data)

        # Compute the objective functions
        execute_energyplan_spool([f"input{i}.txt" for i in range(len(x))])

        # Retrieve CO2 emissions and total annual costs
        co2, variable_cost, operations_cost = find_values(
            ENERGYPLAN_RESULTS,
            "CO2-emission (corrected)",
            "Variable costs",
            "Fixed operation costs"
        ).T

        # Retrieve:
        PV = 0  # annual PV electricity
        HYDRO = 1  # annual hydropower
        WAVE = 2  # annual wave power
        IMPORT = 3  # annual import
        EXPORT = 4  # annual export
        HP = 5  # annual HP electricity
        HH_CHP = 6  # annual HH electricity CHP
        DEMAND = 7  # annual demand
        NGAS = 8  # annual natural gas
        OIL = 9  # annual oil
        BIOGAS = 10  # annual biomass

        z = np.zeros((11, len(x)))

        for i, res in enumerate(ENERGYPLAN_RESULTS.glob("*.txt")):
            D = parse_output(res)
            annual_lbl = [i for i in D.keys() if 'TOTAL FOR ONE YEAR' in i][0]
            fuel_lbl = [i for i in D.keys() if 'ANNUAL FUEL' in i][0]
            z[HYDRO, i] = float(D[annual_lbl]["Hydro Electr."])
            z[PV, i] = float(D[annual_lbl]["PV Electr."])
            z[WAVE, i] = float(max(D[annual_lbl]["Wave Electr."]))
            z[IMPORT, i] = float(D[annual_lbl]["Import Electr."])
            z[EXPORT, i] = float(D[annual_lbl]["Export Electr."])
            z[HH_CHP, i] = float(D[annual_lbl]["HH-CHP Electr."])
            z[HP, i] = float(D[annual_lbl]["HH-HP Electr."])
            z[DEMAND, i] = float(D[annual_lbl]["Electr. Demand"])
            z[NGAS, i] = float(D[fuel_lbl]['TOTAL']["Ngas Consumption"])
            z[OIL, i] = float(D[fuel_lbl]['TOTAL']["Oil Consumption"])
            z[BIOGAS, i] = float(D[fuel_lbl]['TOTAL']["Biomass Consumption"])

        total_additional_cost = (
            z[HYDRO] + z[PV] + z[WAVE] + z[HH_CHP] + z[IMPORT] - z[EXPORT]
        ) * self.addtionalCostPerGWhinKEuro

        HP_capacity = (self.maxHeatDemandInDistribution * percentages[HP] *
                       self.totalHeatDemand * 1e6) / \
                        (self.COP * self.sumOfAllHeatDistributions)

        geo_borehole_cost = (HP_capacity * self.geoBoreholeCostInKWe) / \
            (1 - (1 + self.interest) ** -self.geoBoreHoleLifeTime)

        biomass_boiler_capacity = \
            (self.totalHeatDemand * percentages[BIOMASS]) * \
                1e6 * 1.5 / self.sumOfAllHeatDistributions

        investment_cost_reduction_biomass_boiler = np.where(
            biomass_boiler_capacity > self.currentIndvBiomassBoilerCapacity,
            (self.currentIndvBiomassBoilerCapacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                    (1 - (1 + self.interest) ** -self.boilerLifeTime),
            (biomass_boiler_capacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                        (1 - (1 + self.interest) ** -self.boilerLifeTime)
        )

        # Since OIL has been overwritten, we use directly the index 1 for OIL
        oil_boiler_capacity = (self.totalHeatDemand * percentages[1]) * \
            1e6 * 1.5 / self.sumOfAllHeatDistributions

        investment_cost_reduction_oil_boiler = np.where(
            oil_boiler_capacity > self.currentIndvOilBoilerCapacity,
            (self.currentIndvOilBoilerCapacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                    (1 - (1 + self.interest) ** -self.boilerLifeTime),
            (oil_boiler_capacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                        (1 - (1 + self.interest) ** -self.boilerLifeTime)
        )

        lpg_boiler_capacity = (self.totalHeatDemand * percentages[LPG]) * \
            1e6 * 1.5 / self.sumOfAllHeatDistributions
        
        investment_cost_reduction_lpg_boiler = np.where(
            lpg_boiler_capacity > self.currentIndvLPGBoilerCapacity,
            (self.currentIndvLPGBoilerCapacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                    (1 - (1 + self.interest) ** -self.boilerLifeTime),
            (lpg_boiler_capacity * \
                self.individualBoilerInvestmentCostInKEuro * \
                    self.interest) / \
                        (1 - (1 + self.interest) ** -self.boilerLifeTime)
        )

        reduction_investment_cost = \
            (self.currentPVCapacity * self.PVInvestmentCostInKEuro *
             self.interest) / (1 - (1 + self.interest) ** -self.PVLifeTime) + \
            (self.currentHydroCapacity * self.hydroInvestmentCostInKEuro *
             self.interest) / (1 - (1 + self.interest) ** -self.HydroLifeTime) + \
            (self.currentBiogasCapacity * self.BiogasInvestmentCostInKEuro *
             self.interest) / (1 - (1 + self.interest) ** -self.BiogasLifeTime) + \
            investment_cost_reduction_biomass_boiler + \
            investment_cost_reduction_oil_boiler + \
            investment_cost_reduction_lpg_boiler

        # Retrieve annual investment costs
        annual_investment_costs = np.reshape(find_values(
            ENERGYPLAN_RESULTS,
            "Annual Investment costs"
        ), -1)

        # Compute the real invertment cost
        real_investment_cost = annual_investment_costs - \
            reduction_investment_cost + geo_borehole_cost

        # Compute the actual annual cost, which is the third objective
        actual_annual_cost = variable_cost + operations_cost + \
            total_additional_cost + real_investment_cost

        # Individual house HP electric demand
        individual_house_HP_demand = \
            (z[IMPORT] + z[EXPORT]) / (z[DEMAND] - z[HP])
        
        total_PE_electricity = z[PV] * self.PVPEF + z[HYDRO] * self.HYPEF + \
            z[WAVE] * self.BioGasPEF + z[BIOMASS] * self.BiomassPEF

        total_local_electricity = z[PV] + z[HYDRO] + z[WAVE] + z[HH_CHP]

        total_PE_consumption = total_PE_electricity / total_local_electricity

        ESD = (z[IMPORT] * self.PEFImport + z[OIL] + z[NGAS]) / \
            total_PE_consumption

        out["F"] = np.column_stack([
            co2, actual_annual_cost, individual_house_HP_demand, ESD
        ])
```

In [3]:
from pymoo.optimize import minimize

res = minimize(
    model,
    algorithm,
    ('n_gen', 50),
    seed=5230,
    verbose=True,
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     95 |             - |             -
     2 |      200 |    100 |  0.0088716577 |         ideal
     3 |      300 |    100 |  0.0046565774 |         ideal
     4 |      400 |    100 |  0.0614033764 |         nadir
     5 |      500 |    100 |  0.0161940183 |         ideal
     6 |      600 |    100 |  0.0061386563 |         nadir
     7 |      700 |    100 |  0.0061765722 |         nadir
     8 |      800 |    100 |  0.0102476291 |         nadir
     9 |      900 |    100 |  0.0369542266 |         nadir
    10 |     1000 |    100 |  0.0027941580 |         nadir
    11 |     1100 |    100 |  0.0316112742 |         nadir
    12 |     1200 |    100 |  0.0151684022 |             f
    13 |     1300 |    100 |  0.0049794487 |         nadir
    14 |     1400 |    100 |  0.0034426958 |         ideal
    15 |     1500 |    100 |  0.0157814742 |         nadir
    16 |     1600 |    100 |  0.0175666199 |            

## Results analysis

In [4]:
import pandas as pd
import plotly.express as px

# Load reference data from the paper, no index column and space separated
ref = pd.read_csv('giudicarie-reference.csv', sep=' ', header=None,
                  names=['CO2 emissions (corrected) [Mton]', 'Cost [kEUR]',
                         'LFC', 'ESD'],
                  index_col=False)

# Cast results to DataFrame
df = pd.DataFrame(
    ref,
    columns=[
        'CO2 emissions (corrected) [Mton]',
        'Cost [kEUR]',
        'LFC',
        'ESD'
    ]
) 
# Define plot
fig = px.scatter_3d(
    df,
    x='CO2 emissions (corrected) [Mton]',
    y='Cost [kEUR]',
    z='LFC',
    color='ESD',
    height=800
)
# Show plot
fig.show()

In [5]:

# Get the pareto-front
F = res.F

# Cast results to DataFrame
df = pd.DataFrame(
    F,
    columns=[
        'CO2 emissions (corrected) [Mton]',
        'Cost [kEUR]',
        'LFC',
        'ESD'
    ]
) 
# Define plot
fig = px.scatter_3d(
    df,
    x='CO2 emissions (corrected) [Mton]',
    y='Cost [kEUR]',
    z='LFC',
    color='ESD',
    height=800
)
# Show plot
fig.show()


### Convergence analysis

The results of the paper {cite:t}`MAHBUB2016140` are used as reference to
measure the quality of the solution. We implement the Inverted Generational
Distance (IGD) {cite:t}`COELLOCOELLO2004688` to quantify the distance from any
point in the set of solutions $Z$ to the closest point in the set of
reference solutions $A$.

$$
IGD(A) = \frac{1}{|Z|} \left( \sum_{i=1}^{|Z|} \hat{d}_i ^{\,p} \right) ^{1/p}
$$

where $\hat{d}_i$ represents the Euclidean distance ($p=2$) from $z_i$ to its
nearest reference point in $A$.

The lower the value of the IGD, the closer the set $A$ to the reference set
$Z$.

In [6]:
from pymoo.indicators.igd import IGD

ind = IGD(res.F)
print("IGD", ind(ref.values))

IGD 1070.1242082527372


## References

```{bibliography}
:style: unsrt
```